# Load Data

In [44]:
import pandas as pd 

# columns correspond to unitigs so we must transpose this table
def load_unitig_data(path) -> pd.DataFrame:
    sr = pd.read_csv(path, sep=' ')
    sr.set_index('pattern_id',inplace=True)
    return sr.T[1:]

azm_sr = load_unitig_data("azm_sr_gwas_filtered_unitigs.Rtab")
cfx_sr = load_unitig_data("cfx_sr_gwas_filtered_unitigs.Rtab")
cip_sr = load_unitig_data("cip_sr_gwas_filtered_unitigs.Rtab")

metadata = pd.read_csv('metadata.csv')
metadata.set_index('Sample_ID',inplace=True)


In [45]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3786 entries, ERR1549286 to ERR2172354
Data columns (total 30 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Year            3536 non-null   float64
 1   Country         3785 non-null   object 
 2   Continent       3785 non-null   object 
 3   Beta.lactamase  1927 non-null   object 
 4   Azithromycin    3480 non-null   object 
 5   Ciprofloxacin   3129 non-null   object 
 6   Ceftriaxone     3436 non-null   object 
 7   Cefixime        3405 non-null   object 
 8   Tetracycline    1472 non-null   object 
 9   Penicillin      1465 non-null   object 
 10  NG_MAST         3779 non-null   object 
 11  Group           3786 non-null   int64  
 12  azm_mic         3478 non-null   float64
 13  cip_mic         3088 non-null   float64
 14  cro_mic         3434 non-null   float64
 15  cfx_mic         3401 non-null   float64
 16  tet_mic         1472 non-null   float64
 17  pen_mic         1465 no

In [46]:
metadata.head(5)

,Year,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,...,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cro_sr,cfx_sr,tet_sr,pen_sr
Sample_ID,,,,,,,,,,,,,,,,,,,,,
ERR1549286,2015.0,UK,Europe,NaN,>256,NaN,0.016,NaN,NaN,NaN,...,-5.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549290,2015.0,UK,Europe,NaN,>256,NaN,0.004,NaN,NaN,NaN,...,-7.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549291,2015.0,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,...,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549287,2015.0,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,...,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549288,2015.0,UK,Europe,NaN,>256,NaN,0.008,NaN,NaN,NaN,...,-6.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN


    # Null/NA Cleaning in Labels

1. Remove rows with NaN in the labels we are trying to predict: 'azm_sr','cfx_sr', 'cip_sr'

---

Note to Jacob:
If I remove all rows with nulls, then we drop down to ~1k entries. Not ideal. I will remove only the nulls in the target labels. By doing that I was able to preserve ~2800 entries.

Additionally, I can technically replace the nulls in the feature set with averages whether they are continuous or discrete, but since we are going to be building some kind of predictive model later, it would be bad practice to run column averages in df.fillna() before we split our dataset into training and test sets.

Therefore, I am going to split the dataset into a training and test set first.

What do you think?

-Jacob
We could evaluate how to fill the NaN values based on how the data is skewed for each feature we want to predict. 
if skew > 0 -> fill NaN with mean .. there are more 0s than 1s, mean will reflect the distribution of 0s and 1s
else if skew < 0 -> fill NaN with median .. there are more 1s than 0s, using the median ensures our negatively skewed data wont be affected by an uneven distribution of 0s and 1s when we fill it


In [47]:

def impute_cols_by_skew(df, columns):
    for column_name in columns:
        sr_skew = df[column_name].skew()
        impute_value = None
        if sr_skew > 0:
            impute_value = df[column_name].mean()
        elif sr_skew < 0:
            impute_value = df[column_name].median()
        df[column_name].fillna(impute_value, inplace=True)

#metadata.dropna(axis=0, how='any', inplace=True, subset=['azm_sr', 'cfx_sr', 'cip_sr'])

It looks like they're all positively skewed so we can use mean values, lets try continuous for now and see what happens. It was worth trying this though

In [48]:
metadata.head(5)

,Year,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,...,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cro_sr,cfx_sr,tet_sr,pen_sr
Sample_ID,,,,,,,,,,,,,,,,,,,,,
ERR1549286,2015.0,UK,Europe,NaN,>256,NaN,0.016,NaN,NaN,NaN,...,-5.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549290,2015.0,UK,Europe,NaN,>256,NaN,0.004,NaN,NaN,NaN,...,-7.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549291,2015.0,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,...,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549287,2015.0,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,...,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549288,2015.0,UK,Europe,NaN,>256,NaN,0.008,NaN,NaN,NaN,...,-6.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN


# Removing Un-needed Columns
2. Removing 'Year', and unimportant labels 'cro_sr', 'tet_sr', 'pen_sr'

In [49]:
useless_columns = ['Year', 'cro_sr', 'tet_sr', 'pen_sr']

metadata.drop(labels=useless_columns, axis=1, inplace=True)

In [50]:
metadata.columns

Index(['Country', 'Continent', 'Beta.lactamase', 'Azithromycin',
       'Ciprofloxacin', 'Ceftriaxone', 'Cefixime', 'Tetracycline',
       'Penicillin', 'NG_MAST', 'Group', 'azm_mic', 'cip_mic', 'cro_mic',
       'cfx_mic', 'tet_mic', 'pen_mic', 'log2_azm_mic', 'log2_cip_mic',
       'log2_cro_mic', 'log2_cfx_mic', 'log2_tet_mic', 'log2_pen_mic',
       'azm_sr', 'cip_sr', 'cfx_sr'],
      dtype='object')

# Cleaning non-numeric entries in numeric fields to NaN.

3. Turn Non Numeric Entries in Numeric Columns into NaN
4. Cast all numeric rows into float32
---


Notes: Turning them into NaN for now. Will engineer values for all NaNs after train and test splits are made

In [51]:
numeric_columns = [
'Azithromycin',
'Ciprofloxacin',
'Ceftriaxone',
'Cefixime',
'Tetracycline',
'Penicillin',
'NG_MAST',
'Group',
'azm_mic',
'cip_mic',
'cro_mic',
'cfx_mic',
'tet_mic',
'pen_mic',
'log2_azm_mic',
'log2_cip_mic',
'log2_cro_mic',
'log2_cfx_mic',
'log2_tet_mic',
'log2_pen_mic',
'azm_sr',
'cip_sr',
'cfx_sr',
]

for column in numeric_columns:
    metadata[column] = pd.to_numeric(metadata[column], errors='coerce', downcast="float") #https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_numeric.html

In [52]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3786 entries, ERR1549286 to ERR2172354
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country         3785 non-null   object 
 1   Continent       3785 non-null   object 
 2   Beta.lactamase  1927 non-null   object 
 3   Azithromycin    3373 non-null   float32
 4   Ciprofloxacin   3069 non-null   float32
 5   Ceftriaxone     3432 non-null   float32
 6   Cefixime        3366 non-null   float32
 7   Tetracycline    1470 non-null   float32
 8   Penicillin      1461 non-null   float32
 9   NG_MAST         3238 non-null   float32
 10  Group           3786 non-null   float32
 11  azm_mic         3478 non-null   float32
 12  cip_mic         3088 non-null   float32
 13  cro_mic         3434 non-null   float32
 14  cfx_mic         3401 non-null   float32
 15  tet_mic         1472 non-null   float32
 16  pen_mic         1465 non-null   float32
 17  log2_azm_mic    3478 no

In [53]:
metadata.head(10)

,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,NG_MAST,...,pen_mic,log2_azm_mic,log2_cip_mic,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cfx_sr
Sample_ID,,,,,,,,,,,,,,,,,,,,,
ERR1549286,UK,Europe,NaN,NaN,NaN,0.016,NaN,NaN,NaN,9768.0,...,NaN,9.0,NaN,-5.965784,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549290,UK,Europe,NaN,NaN,NaN,0.004,NaN,NaN,NaN,9768.0,...,NaN,9.0,NaN,-7.965784,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549291,UK,Europe,NaN,NaN,NaN,0.006,NaN,NaN,NaN,9768.0,...,NaN,9.0,NaN,-7.380822,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549287,UK,Europe,NaN,NaN,NaN,0.006,NaN,NaN,NaN,9768.0,...,NaN,9.0,NaN,-7.380822,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549288,UK,Europe,NaN,NaN,NaN,0.008,NaN,NaN,NaN,9768.0,...,NaN,9.0,NaN,-6.965784,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549299,UK,Europe,NaN,NaN,NaN,0.012,NaN,NaN,NaN,NaN,...,NaN,9.0,NaN,-6.380822,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549292,UK,Europe,NaN,NaN,NaN,0.023,NaN,NaN,NaN,9768.0,...,NaN,9.0,NaN,-5.442222,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549298,UK,Europe,NaN,0.5,NaN,0.094,NaN,NaN,NaN,NaN,...,NaN,-1.0,NaN,-3.411196,NaN,NaN,NaN,0.0,NaN,NaN
ERR1549296,UK,Europe,NaN,0.5,NaN,0.094,NaN,NaN,NaN,NaN,...,NaN,-1.0,NaN,-3.411196,NaN,NaN,NaN,0.0,NaN,NaN


# One Hot Encode Categorical Columns

5. Turn Categorical Location entries into numerical representation https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html
6. Handle 'Beta.lactamase' special case, as it is discrete with many NaNs. Cannot be engineered in the same way as continuous features. One hot encode like the other discrete features but set to all 0 if NaN

In [54]:
geographic_columns=['Country', 'Continent']
metadata = pd.get_dummies(data=metadata, prefix="Encoded", columns=geographic_columns, dtype=float)
metadata = pd.get_dummies(data=metadata, prefix="Encoded_Beta.lactamase", columns=['Beta.lactamase'], dtype=float)


In [55]:
metadata.head(5)

,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,NG_MAST,Group,azm_mic,cip_mic,...,Encoded_Africa,Encoded_America,Encoded_Asia,Encoded_Europe,Encoded_Oceania,Encoded_Beta.lactamase_0,Encoded_Beta.lactamase_1,Encoded_Beta.lactamase_2,Encoded_Beta.lactamase_R,Encoded_Beta.lactamase_S
Sample_ID,,,,,,,,,,,,,,,,,,,,,
ERR1549286,NaN,NaN,0.016,NaN,NaN,NaN,9768.0,1058.0,512.0,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR1549290,NaN,NaN,0.004,NaN,NaN,NaN,9768.0,1058.0,512.0,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR1549291,NaN,NaN,0.006,NaN,NaN,NaN,9768.0,1058.0,512.0,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR1549287,NaN,NaN,0.006,NaN,NaN,NaN,9768.0,1058.0,512.0,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR1549288,NaN,NaN,0.008,NaN,NaN,NaN,9768.0,1058.0,512.0,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3786 entries, ERR1549286 to ERR2172354
Data columns (total 98 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Azithromycin              3373 non-null   float32
 1   Ciprofloxacin             3069 non-null   float32
 2   Ceftriaxone               3432 non-null   float32
 3   Cefixime                  3366 non-null   float32
 4   Tetracycline              1470 non-null   float32
 5   Penicillin                1461 non-null   float32
 6   NG_MAST                   3238 non-null   float32
 7   Group                     3786 non-null   float32
 8   azm_mic                   3478 non-null   float32
 9   cip_mic                   3088 non-null   float32
 10  cro_mic                   3434 non-null   float32
 11  cfx_mic                   3401 non-null   float32
 12  tet_mic                   1472 non-null   float32
 13  pen_mic                   1465 non-null   float32
 14

# Split Dataframe into Train and Test

7. To ensure that we can do normalization later as it is good practice to split before normalization

In [57]:
from sklearn.model_selection import train_test_split
train_inputs,test_inputs = train_test_split(metadata, test_size=0.20, random_state=42)   # partition and mix entries
print(train_inputs.shape,":", test_inputs.shape)

(2271, 98) : (1515, 98)


# Engineer/Impute Values for NaN numerical values

8. Fill out gaps so we have more data by avoiding dropping all rows with NaN

In [58]:
# apply skew based imputation

targets = numeric_columns
impute_cols_by_skew(train_inputs, targets)
impute_cols_by_skew(test_inputs, targets)




/tmp/ipykernel_1513/1940559347.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column_name].fillna(impute_value, inplace=True)


In [59]:
train_inputs.head(5)

,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,NG_MAST,Group,azm_mic,cip_mic,...,Encoded_Africa,Encoded_America,Encoded_Asia,Encoded_Europe,Encoded_Oceania,Encoded_Beta.lactamase_0,Encoded_Beta.lactamase_1,Encoded_Beta.lactamase_2,Encoded_Beta.lactamase_R,Encoded_Beta.lactamase_S
Sample_ID,,,,,,,,,,,,,,,,,,,,,
15335_5#74,1.000,8.00000,0.015,0.061716,7.192784,2.604292,1407.0,127.0,1.000,8.00000,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
16853_3#19,0.094,0.00400,0.002,0.016000,7.192784,2.604292,10676.0,462.0,0.094,0.00400,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10625_6#18,0.750,32.00000,0.094,0.190000,2.000000,2.000000,7328.0,390.0,0.750,32.00000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10_062,16.000,7.12129,0.047,0.125000,7.192784,2.604292,3806.0,840.0,16.000,7.46714,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8727_5#86,0.500,32.00000,0.030,0.060000,2.000000,2.000000,1978.0,167.0,0.500,32.00000,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# Normalize Numerical Features 

9. Scale all numerical columns' values to be between 0 and 1

In [60]:
#from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler

normalizer = MinMaxScaler()

train_inputs[numeric_columns] = normalizer.fit_transform(train_inputs[numeric_columns])
test_inputs[numeric_columns] = normalizer.fit_transform(test_inputs[numeric_columns])

train_inputs.head(5)

,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,NG_MAST,Group,azm_mic,cip_mic,...,Encoded_Africa,Encoded_America,Encoded_Asia,Encoded_Europe,Encoded_Oceania,Encoded_Beta.lactamase_0,Encoded_Beta.lactamase_1,Encoded_Beta.lactamase_2,Encoded_Beta.lactamase_R,Encoded_Beta.lactamase_S
Sample_ID,,,,,,,,,,,,,,,,,,,,,
15335_5#74,0.003848,0.124973,0.007376,0.020245,0.028036,0.020331,0.095937,0.118030,0.000973,0.124973,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
16853_3#19,0.000309,0.000031,0.000875,0.005002,0.028036,0.020331,0.728849,0.429368,0.000088,0.000031,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10625_6#18,0.002871,0.499984,0.046881,0.063021,0.007750,0.015610,0.500239,0.362454,0.000729,0.499984,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10_062,0.062445,0.111242,0.023378,0.041347,0.028036,0.020331,0.259747,0.780669,0.015621,0.116646,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8727_5#86,0.001895,0.499984,0.014877,0.019673,0.007750,0.015610,0.134927,0.155204,0.000484,0.499984,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### Data Stats

In [61]:
j=[0,0,0]
samples = metadata.index
for sample in samples:
    if metadata['azm_sr'][sample]:
        j[0] +=1
    if metadata['cfx_sr'][sample]:
        j[1] +=1
    if metadata['cip_sr'][sample]:
        j[2] +=1

print(j[0]/len(samples), "% of samples have resistance to azm")
print(j[1]/len(samples), "% of samples have resistance to cfx")
print(j[2]/len(samples), "% of samples have resistance to cip")


0.199418911780243 % of samples have resistance to azm
0.10301109350237718 % of samples have resistance to cfx
0.5615425250924458 % of samples have resistance to cip


In [62]:
samples = azm_sr.index

# some random unitig from azm_sr

from random import randint
randomUnitig = azm_sr.columns[randint(0,azm_sr.shape[1])]
j = 0
print(azm_sr.shape)
for i in range(azm_sr.shape[0]):
    if azm_sr[randomUnitig][samples[i]]:
        j += 1

print(randomUnitig,"\npresent in", j/azm_sr.shape[0],"% of azm_sr samples (",j,"/",azm_sr.shape[0],')' )

(3970, 515)
CGGCATCCGATATGAACTCCTCGCAACGCAAA 
present in 0.3410579345088161 % of azm_sr samples ( 1354 / 3970 )


In [64]:
train_inputs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2271 entries, 15335_5#74 to 15335_4#72
Data columns (total 98 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Azithromycin              2271 non-null   float32
 1   Ciprofloxacin             2271 non-null   float32
 2   Ceftriaxone               2271 non-null   float32
 3   Cefixime                  2271 non-null   float32
 4   Tetracycline              2271 non-null   float32
 5   Penicillin                2271 non-null   float32
 6   NG_MAST                   2271 non-null   float32
 7   Group                     2271 non-null   float32
 8   azm_mic                   2271 non-null   float32
 9   cip_mic                   2271 non-null   float32
 10  cro_mic                   2271 non-null   float32
 11  cfx_mic                   2271 non-null   float32
 12  tet_mic                   2271 non-null   float32
 13  pen_mic                   2271 non-null   float32
 14

In [65]:
test_inputs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1515 entries, SRR3360667 to 17428_6#79
Data columns (total 98 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Azithromycin              1515 non-null   float32
 1   Ciprofloxacin             1515 non-null   float32
 2   Ceftriaxone               1515 non-null   float32
 3   Cefixime                  1515 non-null   float32
 4   Tetracycline              1515 non-null   float32
 5   Penicillin                1515 non-null   float32
 6   NG_MAST                   1515 non-null   float32
 7   Group                     1515 non-null   float32
 8   azm_mic                   1515 non-null   float32
 9   cip_mic                   1515 non-null   float32
 10  cro_mic                   1515 non-null   float32
 11  cfx_mic                   1515 non-null   float32
 12  tet_mic                   1515 non-null   float32
 13  pen_mic                   1515 non-null   float32
 14